<a href="https://colab.research.google.com/github/JyothilalithaP/SafeSenior/blob/main/SafeSenior.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pandas scikit-learn imbalanced-learn xgboost
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from datetime import datetime

# HEALTH MONITORING SYSTEM

# Load health data
health_df = pd.read_csv('health_monitoring.csv')

# Preprocess health data
health_df['Alert Triggered (Yes/No)'] = health_df['Alert Triggered (Yes/No)'].map({'Yes': 1, 'No': 0})
health_df[['Systolic BP', 'Diastolic BP']] = health_df['Blood Pressure'].str.extract(r'(\d+)\/(\d+)').astype(int)

# Define health features
health_features = ['Heart Rate', 'Systolic BP', 'Diastolic BP', 'Glucose Levels', 'Oxygen Saturation (SpO₂%)']
X_health = health_df[health_features]
y_health = health_df['Alert Triggered (Yes/No)']

# Scale health data
health_scaler = StandardScaler()
X_health_scaled = health_scaler.fit_transform(X_health)

# Train health model
X_health_train, X_health_test, y_health_train, y_health_test = train_test_split(
    X_health_scaled, y_health, test_size=0.2, random_state=42
)
rf_health_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_health_model.fit(X_health_train, y_health_train)

# SAFETY MONITORING SYSTEM

# Load safety data
safety_df = pd.read_csv('safety_monitoring.csv')

# Preprocess safety data
safety_df['Fall Detected (Yes/No)'] = safety_df['Fall Detected (Yes/No)'].map({'Yes': 1, 'No': 0})
impact_map = {'-': 0, 'Low': 1, 'Medium': 2, 'High': 3}
safety_df['Impact Force'] = safety_df['Impact Force Level'].map(impact_map).fillna(0)

# Define safety features
safety_features = ['Movement Activity', 'Impact Force', 'Post-Fall Inactivity Duration (Seconds)', 'Location']
X_safety = safety_df[safety_features]
y_safety = safety_df['Fall Detected (Yes/No)']

# Create safety preprocessor
safety_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Impact Force', 'Post-Fall Inactivity Duration (Seconds)']),
        ('cat', OneHotEncoder(), ['Movement Activity', 'Location'])
    ])

# Process and balance safety data
X_safety_processed = safety_preprocessor.fit_transform(X_safety)
X_safety_train, X_safety_test, y_safety_train, y_safety_test = train_test_split(
    X_safety_processed, y_safety, test_size=0.2, random_state=42, stratify=y_safety
)

smote = SMOTE(random_state=42)
X_safety_res, y_safety_res = smote.fit_resample(X_safety_train, y_safety_train)

# Train safety model
rf_safety_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_safety_model.fit(X_safety_res, y_safety_res)

# REMINDER SYSTEM

# Load reminder data
reminder_df = pd.read_csv('daily_reminder.csv')

# Preprocess reminder data
reminder_df['Acknowledged'] = reminder_df['Acknowledged (Yes/No)'].map({'Yes': 1, 'No': 0})
reminder_df['Scheduled Hour'] = pd.to_datetime(reminder_df['Scheduled Time'], format='%H:%M:%S').dt.hour

# Define reminder features
reminder_features = ['Reminder Type', 'Scheduled Hour']
X_reminder = reminder_df[reminder_features]
y_reminder = reminder_df['Acknowledged']

# Create reminder encoder
reminder_encoder = OneHotEncoder(handle_unknown='ignore')
X_reminder_encoded = reminder_encoder.fit_transform(X_reminder)

# Train reminder model
X_reminder_train, X_reminder_test, y_reminder_train, y_reminder_test = train_test_split(
    X_reminder_encoded, y_reminder, test_size=0.2, random_state=42
)
rf_reminder_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_reminder_model.fit(X_reminder_train, y_reminder_train)

# INTEGRATED SYSTEM

class CompleteElderlyMonitor:
    def __init__(self, health_model, safety_model, reminder_model,
                 health_scaler, safety_preprocessor, reminder_encoder):
        self.health_model = health_model
        self.safety_model = safety_model
        self.reminder_model = reminder_model
        self.health_scaler = health_scaler
        self.safety_preprocessor = safety_preprocessor
        self.reminder_encoder = reminder_encoder

    def assess_complete_status(self, health_data, safety_data, reminder_data):
        # Health assessment
        health_df = pd.DataFrame([health_data], columns=health_features)
        health_processed = self.health_scaler.transform(health_df)
        health_alert = self.health_model.predict(health_processed)[0]

        # Safety assessment
        safety_df = pd.DataFrame([safety_data], columns=safety_features)
        safety_processed = self.safety_preprocessor.transform(safety_df)
        safety_alert = self.safety_model.predict(safety_processed)[0]

        # Reminder assessment
        reminder_hour = pd.to_datetime(reminder_data['time']).hour
        reminder_input = pd.DataFrame([[reminder_data['type'], reminder_hour]],
                                    columns=['Reminder Type', 'Scheduled Hour'])
        reminder_processed = self.reminder_encoder.transform(reminder_input)
        reminder_ack_prob = self.reminder_model.predict_proba(reminder_processed)[0][1]

        # Comprehensive report
        print("\n=== COMPREHENSIVE ELDERLY STATUS REPORT ===")
        print(f"Health Status: {'CRITICAL' if health_alert else 'Normal'}")
        print(f"Safety Status: {'FALL DETECTED' if safety_alert else 'Normal'}")
        print(f"Reminder Acknowledgment Probability: {reminder_ack_prob:.2%}")

        if health_alert or safety_alert:
            print("\n🚨 ALERT: Immediate attention required!")
        elif reminder_ack_prob < 0.3:
            print("\n⚠️ Warning: Low reminder acknowledgment likelihood")
        else:
            print("\n✅ All systems normal")

# Initialize complete system
complete_monitor = CompleteElderlyMonitor(
    health_model=rf_health_model,
    safety_model=rf_safety_model,
    reminder_model=rf_reminder_model,
    health_scaler=health_scaler,
    safety_preprocessor=safety_preprocessor,
    reminder_encoder=reminder_encoder
)

In [ ]:
# Example 1: Morning medication check with normal vitals
print("\n=== EXAMPLE 1: MORNING MEDICATION CHECK ===")
morning_health = [68, 118, 78, 95, 97]  # Excellent vitals
morning_safety = ['Sitting', 0, 0, 'Kitchen']  # Having breakfast
morning_reminder = {'type': 'Medication', 'time': '08:30:00'}  # Morning meds
complete_monitor.assess_complete_status(morning_health, morning_safety, morning_reminder)

# Example 2: Afternoon exercise with slightly elevated vitals
print("\n=== EXAMPLE 2: AFTERNOON EXERCISE ===")
afternoon_health = [85, 130, 85, 105, 95]  # Slightly elevated from exercise
afternoon_safety = ['Walking', 0, 0, 'Living Room']  # Walking around
afternoon_reminder = {'type': 'Exercise', 'time': '15:00:00'}  # Daily exercise
complete_monitor.assess_complete_status(afternoon_health, afternoon_safety, afternoon_reminder)

# Example 3: Evening potential fall scenario
print("\n=== EXAMPLE 3: EVENING FALL DETECTION ===")
evening_health = [92, 140, 90, 98, 94]  # Elevated from stress
evening_safety = ['No Movement', 2, 180, 'Bathroom']  # Possible fall in bathroom
evening_reminder = {'type': 'Hydration', 'time': '20:00:00'}  # Evening water reminder
complete_monitor.assess_complete_status(evening_health, evening_safety, evening_reminder)

# Example 4: Nighttime check with missed medication
print("\n=== EXAMPLE 4: NIGHTTIME CHECK ===")
night_health = [72, 125, 80, 150, 96]  # Elevated glucose (missed meds)
night_safety = ['Lying', 0, 0, 'Bedroom']  # Sleeping
night_reminder = {'type': 'Medication', 'time': '22:00:00'}  # Missed night meds
complete_monitor.assess_complete_status(night_health, night_safety, night_reminder)


=== EXAMPLE 1: MORNING MEDICATION CHECK ===

=== COMPREHENSIVE ELDERLY STATUS REPORT ===
Health Status: Normal
Safety Status: Normal
Reminder Acknowledgment Probability: 46.56%

✅ All systems normal

=== EXAMPLE 2: AFTERNOON EXERCISE ===

=== COMPREHENSIVE ELDERLY STATUS REPORT ===
Health Status: Normal
Safety Status: Normal
Reminder Acknowledgment Probability: 39.34%

✅ All systems normal

=== EXAMPLE 3: EVENING FALL DETECTION ===

=== COMPREHENSIVE ELDERLY STATUS REPORT ===
Health Status: CRITICAL
Safety Status: FALL DETECTED
Reminder Acknowledgment Probability: 38.86%

🚨 ALERT: Immediate attention required!

=== EXAMPLE 4: NIGHTTIME CHECK ===

=== COMPREHENSIVE ELDERLY STATUS REPORT ===
Health Status: CRITICAL
Safety Status: Normal
Reminder Acknowledgment Probability: 37.29%

🚨 ALERT: Immediate attention required!
